In [7]:
M <- matrix(c(c(-2,-1,2,1),c(3,1,-1,-3),c(-1,0,-1,2)), nrow=4)

In [10]:
pca <- function(M, scale=TRUE, bias=FALSE){
    # returned object :
    base_result = list("base"=M)
    
    # fonction qui calcule la matrice de covariance de la matrice initiale et renvoit toutes les étapes précédentes de calcul
    covariance_matrix <- function(M){
        # fonction qui retourne la matrice des données centrées et le centre de gravité de la matrice initiale
        centered_matrix <- function(M){
            # fonction qui calcule la matrice G, centre de gravité de la matrice passée en paramètre
            gravity_center_matrix <- function(M){
                moy_T <- colMeans(M, na.rm = FALSE, dims = 1)
                M_moy_t <- t(matrix(rep(moy_T, dim(M)[1]), ncol = dim(M)[1]))
                return(M_moy_t)
            }
            G <- gravity_center_matrix(M)
            return(list("G"=G, "M_centered"=M-G))
        }
        Mc <- centered_matrix(M)
        M_temp <- t(Mc[["M_centered"]]) %*% Mc[["M_centered"]]
        S_bias <- (1 / dim(M)[1]) * M_temp
        S <- (1 / (dim(M)[1]-1)) * M_temp
        return(append(Mc, list("S"=S, "S_bias"=S_bias)))
    }
    base_result <- append(base_result, covariance_matrix(M))
    
    # fonction de calcul de la matrice de correlation de la matrice initiale et renvoit toutes les étapes précédentes de calcul
    correlation_matrix <- function(S, M_centered){
        # fonction qui calcule la matrice diagonale des (1/écarts-type) des colonnes
        diag_ecart_type <- function(S){
            return(list("D1_div_sd"= (1/sqrt(diag(S))) * diag(dim(S)[1])))
        }
        result <- diag_ecart_type(S)
        
        # fonction qui calcule la matrice centrée réduite
        centered_reduced_matrix <- function(M_centered, D1_S){
            return(list("M_scale"= M_centered %*% D1_S))
        }
        result <- append(result, centered_reduced_matrix(M_centered, result[["D1_div_sd"]]))
        
        # calcul de la matrice de correlation
        R_temp <- t(result[["M_scale"]]) %*% result[["M_scale"]]
        R <- (1 / dim(M)[1]) * R_temp
        return(append(result, list("R"=R)))
    }
    if(scale){
        base_result <- append(base_result, correlation_matrix(base_result[["S_bias"]], base_result[["M_centered"]]))
    }
    if(bias){
        base_result[["S"]] <- base_result[["S_bias"]]
    }
    base_result[["S_bias"]] <- NULL
    
    # building PCA main component
    pca_component <- list()
    Si <- c()
    if(scale){
        Si <- R
    }else{
        Si <- base_result[["S"]]

    }
    pca_component[["Si"]] <- Si
    
    
    return(base_result)
}

In [12]:
print(pca(M, bias=TRUE, scale=TRUE))

ERROR: Error in pca(M, bias = TRUE, scale = TRUE): objet 'R' introuvable


In [7]:
inertia <- function(eigenvalues){
  return(sum(eigenvalues))
}

cumulative_information_caught <- function(eigenvalues, num_axes = 2, percent = T){
  return(sum(eigenvalues[1:num_axes])/inertia(eigenvalues)*((1-(percent*1))+(percent*1)*100))
}

information_caught <- function(eigenvalues, index_axe, percent = T){
  return(eigen_values[index_axe]/inertia(eigenvalues)*((1-(percent*1))+(percent*1)*100))
}

In [13]:
eigen_function <- function(M){
    dimM = dim(M)[1]
    eigen_values <- matrix(0, ncol=dimM)
    eigen_vectors <- matrix(0, ncol=dimM, ncol=dimM)
    
    
    
    for(i in 1:dimM){
        eigen_vectors[i,] <- solve(M-eigen_values[i]*diag(dimM))%*%matrix(0, nrow=dimM)
    }
    
    return(list("eigen_values"=M, "eigen_vectors"=eigen_vectors))
} 

ERROR: Error in M - 2 * diag(3): tableaux de tailles inadéquates
